In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import callbacks
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Input, Dense, Flatten
import openai
import matplotlib.image as img

C:\Users\User\AppData\Local\Temp\ipykernel_30280\442366915.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory ='C:/Users/User/Allergy-Pal/kaggle/input/food41/images',
    image_size = (224, 224),
    validation_split = 0.2,
    subset = "training",
    seed = 42,
    color_mode = 'rgb',
    shuffle = True , 
    label_mode="categorical" , 
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    directory ='C:/Users/User/Allergy-Pal/kaggle/input/food41/images',
    image_size = (224, 224),
    validation_split = 0.2,
    subset = "validation",
    seed = 42,
    color_mode = 'rgb',
    label_mode="categorical" , 
    batch_size=32
)

Found 101000 files belonging to 101 classes.
Using 80800 files for training.
Found 101000 files belonging to 101 classes.
Using 20200 files for validation.


In [3]:
classes = []
with open("C:/Users/User/Allergy-Pal/kaggle/input/food41/meta/meta/classes.txt") as f:
    for line in f.readlines():
        classes.append(line.split("\n")[0]) 

print(classes)

['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheesecake', 'cheese_plate', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobster_roll_sandwich', 'macaroni_and_cheese', 'macarons', 'miso_sou

In [4]:
img_size = (180 , 180)
batch_size = 32

callback_list=[
    callbacks.EarlyStopping(monitor="val_accuracy",patience=10,restore_best_weights=True),
    callbacks.ReduceLROnPlateau(factor=0.8,monitor="val_accuracy",patience=3)
]

In [ ]:
base_model = tf.keras.applications.EfficientNetB0(include_top = False , input_shape=(224 , 224 , 3))  

base_model.trainable = False 

inputs = tf.keras.layers.Input(shape = (224 , 224 , 3) , name = "Input_layer")
x = base_model(inputs)
x = tf.keras.layers.Dense(101)(x)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(101 , 
                                activation = "softmax" , 
                                name = "Output_layer")(x) 
Model_1 = tf.keras.Model(inputs , outputs) 

Model_1.compile(
    loss = tf.keras.losses.categorical_crossentropy ,
    optimizer = tf.keras.optimizers.Adam() , 
    metrics = ['accuracy'] 
)

Model_1_History = Model_1.fit(train_ds 
                              , validation_data = val_ds ,
                              epochs = 6  ,
                              verbose = 1 ,
                              callbacks = callback_list)

In [44]:
Model_1.save("C:/Users/User/Allergy-Pal/model.keras")

img = tf.keras.utils.load_img(
    "C:/Users/User/Downloads/pizza12.jpg", target_size=(224, 224)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) 


predictions = Model_1.predict(img_array)
score = tf.nn.softmax(predictions[0])
print(predictions)
print(np.argmax(predictions[0]))
print(np.max(predictions[0]))

print(
    "This image most likely belongs to {} with a {:.2f}% percent confidence."
    .format(classes[np.argmax(score)], 100 * np.max(predictions[0]))
)

1/1 [==============================] - 1s 1s/step
Predicted Class Index: 3
Confidence: 2.65%
Predicted Class: beef_carpaccio


In [7]:
Model_1 = tf.keras.models.load_model("C:/Users/User/Allergy-Pal/model.keras")

base_model.trainable = True

for layer in base_model.layers[:-10]:
    layer.trainable = False

Model_1.compile(loss = "categorical_crossentropy" ,
                optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001), 
                metrics = ["accuracy"]
               )

initial_epoch = 5

Fine_Tune_epoch = initial_epoch + 5

Stage_2_history = Model_1.fit(train_ds ,
                              epochs = Fine_Tune_epoch , 
                              validation_data = val_ds ,
                              validation_steps = len(val_ds) ,
                              initial_epoch = initial_epoch-1)

Epoch 5/10
2525/2525 [==============================] - 1027s 405ms/step - loss: 0.8292 - accuracy: 0.8861 - val_loss: 8.2527 - val_accuracy: 0.5792
Epoch 6/10
2525/2525 [==============================] - 1450s 574ms/step - loss: 0.5289 - accuracy: 0.9152 - val_loss: 8.0561 - val_accuracy: 0.5788
Epoch 7/10
2525/2525 [==============================] - 1470s 582ms/step - loss: 0.3910 - accuracy: 0.9326 - val_loss: 7.7413 - val_accuracy: 0.5811
Epoch 8/10
2525/2525 [==============================] - 1515s 600ms/step - loss: 0.3214 - accuracy: 0.9429 - val_loss: 7.7662 - val_accuracy: 0.5874
Epoch 9/10
2525/2525 [==============================] - 1095s 434ms/step - loss: 0.2873 - accuracy: 0.9485 - val_loss: 7.7130 - val_accuracy: 0.5907
Epoch 10/10
2525/2525 [==============================] - 1126s 446ms/step - loss: 0.2400 - accuracy: 0.9548 - val_loss: 7.7250 - val_accuracy: 0.5908


In [31]:
Model_1.save("C:/Users/User/Allergy-Pal/model.keras")

img = tf.keras.utils.load_img(
    "C:/Users/User/Downloads/pizza123.jpg", target_size=(224, 224)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) 


predictions = Model_1.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f}% percent confidence."
    .format(classes[np.argmax(score)], 100 * np.max(predictions[0]))
)

1/1 [==============================] - 0s 47ms/step
This image most likely belongs to pizza with a 100.00% percent confidence.
